### Preprocess data

In [1]:
!pip install tensorflow
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 726.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 148.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.2 MB/s eta 0:00:00


In [10]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [12]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
x_train.shape, x_test.shape

((50000, 3072), (10000, 3072))

### NN Classifier

In [5]:
class NN():
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x):
        test_length = x.shape[0]
        y_pred = np.zeros(test_length, dtype=self.y_train.dtype)
        for i in tqdm(range(test_length)):
            distance = np.sum(np.abs(self.x_train - x[i, :]), axis=1)
            min_index = np.argmin(distance)
            y_pred[i] = self.y_train[min_index]
        return y_pred

In [6]:
NN = NN(x_train, y_train)

In [7]:
y_pred = NN.predict(x_test)

  0%|          | 0/10000 [00:00<?, ?it/s]/tmp/ipython-input-933491591.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred[i] = self.y_train[min_index]
 35%|███▌      | 3514/10000 [05:17<09:47, 11.04it/s]/tmp/ipython-input-933491591.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred[i] = self.y_train[min_index]
100%|██████████| 10000/10000 [14:53<00:00, 11.20it/s]


In [8]:
acc = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
f1_score = 2 * (precision * recall) / (precision + recall)

acc, recall, precision, f1_score

(0.2492, 0.2492, 0.2851382838279883, 0.2659605814537101)

### KNN Classifier

In [17]:
class KNN():
    def __init__(self, x_train, y_train, k, classes):
        self.x_train = x_train
        self.y_train = y_train
        self.k = k
        self.classes = classes

    def predict(self, x):
        test_length = x.shape[0]
        y_pred = np.zeros(test_length, dtype=self.y_train.dtype)
        for i in tqdm(range(test_length)):
            distance = np.sqrt(np.sum(np.square(self.x_train - x[i, :]), axis=1))
            index = np.argsort(distance)
            classes = np.zeros(len(self.classes), dtype=np.uint8)
            for j in range(self.k):
                classes[self.y_train[index[j]]] += 1
            y_pred[i] = np.argmax(classes)
        return y_pred

In [14]:
classes = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [19]:
KNN = KNN(x_train, y_train, 5, classes)

In [20]:
x_test_short, y_test_short = x_test[:1000], y_test[:1000]

knn_y_pred = KNN.predict(x_test_short)

100%|██████████| 1000/1000 [01:33<00:00, 10.70it/s]


In [21]:
acc = accuracy_score(y_test_short, knn_y_pred)
recall = recall_score(y_test_short, knn_y_pred, average='macro')
precision = precision_score(y_test_short, knn_y_pred, average='macro')
f1_score = 2 * (precision * recall) / (precision + recall)

acc, recall, precision, f1_score

(0.216, 0.21260384431037452, 0.2823803047847192, 0.24257398328621194)